In [ ]:
%%writefile code.h

int multiply(int a, int b);

Writing code.h


In [ ]:
%%writefile code.cpp

#include "code.h"
using namespace std;

int multiply(int a, int b) {
  return a * b;
};

Writing code.cpp


In [ ]:
%%writefile main.cpp

#include <iostream>
using namespace std
#include "code.h"

int main() {
  cout << multiply(6,7) << '\n';
  return 0;
};

Writing main.cpp


In [ ]:
!g++ main.cpp code.cpp -o main
!./main

code.cpp:3:17: warning: using directive refers to implicitly-defined namespace 'std'
using namespace std;
                ^
1 warning generated.
42


In [23]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [25]:
%%writefile test.cu

#include <cstdio>
#define cudaCheckError() { cudaError_t e=cudaGetLastError(); if(e!=cudaSuccess) printf("CUDA error %s\n", cudaGetErrorString(e)); }

__global__ void helloCUDA() {
    printf("Hello from CUDA kernel cuda!\n");
}

int main() {
    helloCUDA<<<1,1>>>();
    cudaDeviceSynchronize();
    cudaCheckError();
    return 0;
}

Overwriting test.cu


In [27]:
!nvcc -arch=sm_75 test.cu -o test_cuda
!./test_cuda

Hello from CUDA kernel cuda!


In [1]:
%%writefile streaming.cu


#include <cuda_runtime.h>
#include <stdio.h>

__global__ void vectorAdd(const float *A, const float *B, float *C, int numElems) {
    // conditional in place when num threads exceed needed operations.
    // E.g 256 threads/block * 4 blocks = 1024 threads, but only 1000 needed
    // Extra 24 threads will be ignored
    if (threadIdx.x >= numElems) {
        return;
    }
    for (int i = 0; i < numElems; i++) {
      C[i] = A[i] + B[i];
    }
}

int main() {
    // PROBLEM: optimize a large vector addition process using streaming and paralellism

    // currently in realms of host (CPU) so can use non-cuda operations
    int numRows = 50000;
    size_t size = numRows * sizeof(float);
    float *h_A, *h_B, *h_C;

    // Allocate host memory
    h_A = (float *) malloc(size);
    h_B = (float *) malloc(size);
    h_C = (float *) malloc(size);

    // Initialize host arrays
    for (int i = 0; i < numRows; ++i) {
        h_A[i] = rand() / (float)RAND_MAX;
        h_B[i] = rand() / (float)RAND_MAX;
    }

    // Allocate memory for device arrays
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // create streams to async-ly copy data to device
    cudaStream_t stream1, stream2;

    cudaStreamCreate(&stream1);
    cudaStreamCreate(&stream2);

    cudaMemcpyAsync(d_A, h_A, size, cudaMemcpyHostToDevice, stream1);
    cudaMemcpyAsync(d_B, h_B, size, cudaMemcpyHostToDevice, stream2);

    // Launch kernel<<<blocksPerGrid, threadsPerBlock, 0, stream1>>>(d_A, d_B, d_C, numRows);
    int threadsPerBlock = 256;
    int blocksPerGrid = (numRows + threadsPerBlock - 1) / threadsPerBlock;
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(h_A, h_B, h_C, size);


    // copy data back to host
    cudaMemcpyAsync(h_C, d_C, size, cudaMemcpyDeviceToHost, stream1);

    // clean up
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    free(h_A);
    free(h_B);
    free(h_C);

};



Writing streaming.cu


In [2]:
!nvcc -arch=sm_75 streaming.cu -o stream
!./stream

In [85]:
%%writefile image_cv_ops.cu

#include <cuda_runtime.h>
#include <stdio.h>
#include <fstream>
#include <vector>
#include <opencv2/opencv.hpp>
#include <iostream>

__global__ void apply_gaussian_blur(const float *image, float *out_image, int num_pixels) {
    // conditional in place when num threads exceed needed operations.
    if (threadIdx.x >= num_pixels) {
        return;
    }
    int pixel_index = threadIdx.x;

    // create the gaussian kernel
    float kernel[3][3] = {
        {1.0/16, 2.0/16, 1.0/16},
        {2.0/16, 4.0/16, 2.0/16},
        {1.0/16, 2.0/16, 1.0/16}
    };

    // apply blur to image, pasting to out_image
    int kernel_size = 3;
    float sum = 0;
    for (int i = 0; i < kernel_size; i++) {
        for (int j  = 0; j < kernel_size; j++) {
            int image_index = pixel_index + i * kernel_size + j;
            sum += image[image_index] * kernel[i][j];
        }
    }
    out_image[pixel_index] = sum;
}

void run_gaussian_blur(const float *image, float *out_image, int num_pixels) {
    // copy image to VRAM (shared memory)
    size_t size_pixels = num_pixels * sizeof(float);
    float *d_image, *d_out_image;
    cudaMalloc(&d_image, size_pixels);
    cudaMalloc(&d_out_image, size_pixels);
    cudaMemcpy(d_image, image, size_pixels, cudaMemcpyHostToDevice);

    // apply gaussian blur to image
    int threadsPerBlock = 256;
    int blocksPerGrid = (num_pixels + threadsPerBlock - 1) / threadsPerBlock;
    apply_gaussian_blur<<<blocksPerGrid, threadsPerBlock>>>(d_image, d_out_image, num_pixels);

    // copy image back to host
    cudaMemcpy(out_image, d_out_image, size_pixels, cudaMemcpyDeviceToHost);

    // clean up
    cudaFree(d_image);
    cudaFree(d_out_image);
}

int main() {
    // open image using cv::imread()
    cv::Mat image = cv::imread("1b_cat.bmp", cv::IMREAD_GRAYSCALE);
    if (image.empty()) {
        printf("Could not open or find the image\n");
        return -1;
    }

    int num_pixels = image.rows * image.cols;
    float *h_image = (float*) image.data;
    float *h_out_image = (float*) malloc(num_pixels * sizeof(float));

    // apply gaussian blur to image
    run_gaussian_blur(h_image, h_out_image, num_pixels);

    // convert the output image back to cv::Mat
    cv::Mat out_image(image.size(), image.type(), h_out_image);
    cv::imwrite("blurred_image.jpg", out_image);

    // clean up
    free(h_out_image);
    return 0;
}

Overwriting image_cv_ops.cu


In [86]:
!nvcc -arch=sm_75 image_cv_ops.cu -o cv_ops -I/usr/include/opencv4 -w -L/usr/lib -lopencv_core -lopencv_imgcodecs -lopencv_highgui -lopencv_imgproc > /dev/null 2>&1
!./cv_ops